In [33]:
import pandas as pd
import numpy as np
import json
import spacy
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

In [34]:
pd.reset_option('^display.', silent=True)
df = pd.read_csv("../data/textual_similarity_recent.csv")
df.head()

,id,title,body,portal,date_published,features_qids
0,11500787,"Stali tramvaji, pa uskočili autobusi",Električni problemi zaustavili su u utorak neš...,glas-slavonije.hr,2023-03-21 00:00:00,"{188878,2717994,13056097}"
1,11499598,Šetalište Jean-Paula Belmonda,"Most Bir-Hakeim, koji je poznat po željeznim s...",glas-slavonije.hr,2023-03-21 00:00:00,"{243,290355,842778,71567030,109886181}"
2,11499594,U Njemačkoj sve više punionica,Njemačka je u 2022. povećala broj javnih punio...,glas-slavonije.hr,2023-03-21 00:00:00,"{183,1009022,20823045}"
3,11499593,"Aramco, Geely i Renault žele novu tvrtku","Aramco, globalna energetsko-kemijska tvrtka, p...",glas-slavonije.hr,2023-03-21 00:00:00,"{3264974,7427119,18816188,55118127}"
4,11499592,Nevezivanje pojasa i dalje je veliki sigurnosn...,Istraživanje o korištenju sigurnosnog pojasa k...,glas-slavonije.hr,2023-03-21 00:00:00,{}


In [35]:
df.drop_duplicates(subset="body", keep="first", inplace=True)
df.shape

(29625, 6)

## Jaccard coefficient using named entities

In [36]:
df["qids_set"] = df["features_qids"].apply(
    lambda x: set(int(i) for i in x[1:-1].split(",") if i != '')
)

In [37]:
df[["qids_set"]].head()

,qids_set
0,"{13056097, 2717994, 188878}"
1,"{109886181, 290355, 243, 71567030, 842778}"
2,"{20823045, 1009022, 183}"
3,"{18816188, 55118127, 3264974, 7427119}"
4,{}


In [38]:
df.iloc[20]

id                                                         11499555
title                                   Milijardu eura zbog potresa
body              Europska komisija dat će milijardu eura pomoći...
portal                                            glas-slavonije.hr
date_published                                  2023-03-21 00:00:00
features_qids                                    {43,108594,207118}
qids_set                                       {108594, 43, 207118}
Name: 20, dtype: object

In [39]:
#for ind, i in tqdm(df.iterrows()):
#    if len(i["body"]) < 300:
#        print(ind)

In [40]:
set_l = df[["qids_set"]].iloc[20].tolist()[0]
set_l

{43, 108594, 207118}

In [41]:
df["sim1"] = df["qids_set"].apply(
    lambda x: len(x.intersection(set_l))/len(x.union(set_l))
)

In [42]:
pd.set_option('max_colwidth', None)
df.sort_values("sim1", ascending=False)[["title", "body"]][0:15]

,title,body
20,Milijardu eura zbog potresa,"Europska komisija dat će milijardu eura pomoći potresom pogođenoj Turskoj, rekla je u ponedjeljak predsjednica tog tijela Ursula von der Leyen na početku međunarodne donorske konferencije. Von der Leyen je o potresu, koji je odnio gotovo 50 tisuća života u Turskoj i Siriji, govorila kao o najvećoj prirodnoj katastrofi koja je pogodila regiju u posljednjih nekoliko godina. “Kuće, škole i bolnice moraju se ponovno izgraditi, uz najviše standarde sigurnosti od potresa. Voda, kanalizacija i druga ključna infrastruktura moraju se popraviti”, poručila je. Europska komisija dat će dodatnih 108 milijuna eura za humanitarnu pomoć i oporavak Sirije. H"
9569,Iz zgrade koja se srušila nakon niza potresa spasioci izvukli nekoliko ozlijeđenih,"Spasilačke službe u nedjelju su izvukle nekoliko ozlijeđenih osoba iz šesterokatnice u turskome gradu Sanliurfe, koja se srušila nakon oštećenja nastalih na njoj u razornim potresima 6. veljače, piše novinska agencija Anadolija.\nNakon prva dva razorna potresa magnitude 7,7 i 7,6 koji su prije mjesec dana pogodili jugoistočnu Tursku i sjevernu Siriju, uslijedio je niz naknadnih potresa.\nŠesterokatnica u Sanliurfi je oštećena nakon prva dva snažna potresa, no tada se nije srušila. Niz naknadnih potresa utjecao je na statiku zgrade koja se na kraju ipak srušila.\nAgencija prenosi da su zaposlenici hitne pomoći pozvali sve prisutne u blizini da utihnu kako bi uspjeli čuti zvukove koji su dolazili ispod ruševina.\nFotografije snimljene sigurnosnom kamerom pokazale su na koji se način zgrada srušila na prometnu cestu.\nU Turskoj je nakon potresa ostalo gotovo dva milijuna beskućnika. Mnogi su smješteni u šatorske kampove, no dio ljudi se počeo vraćati u svoje oštećene domove."
4684,"Iz ruševina zgrade u Turskoj, koja se srušila nakon niza potresa, spasili su više ljudi","Šesterokatnica u Sanliurfi je oštećena nakon prva dva snažna potresa, no tada se nije srušila. Niz naknadnih potresa utjecao je na statiku zgrade koja se na kraju ipak srušila.\nSpasilačke službe u nedjelju su izvukle nekoliko ozlijeđenih osoba iz šesterokatnice u turskome gradu Sanliurfe, koja se srušila nakon oštećenja nastalih na njoj u razornim potresima 6. veljače, piše novinska agencija Anadolija.\nNakon prva dva razorna potresa magnitude 7,7 i 7,6 koji su prije mjesec dana pogodili jugoistočnu Tursku i sjevernu Siriju, uslijedio je niz naknadnih potresa.\nŠesterokatnica u Sanliurfi je oštećena nakon prva dva snažna potresa, no tada se nije srušila. Niz naknadnih potresa utjecao je na statiku zgrade koja se na kraju ipak srušila.\nAgencija prenosi da su zaposlenici hitne pomoći pozvali sve prisutne u blizini da utihnu kako bi uspjeli čuti zvukove koji su dolazili ispod ruševina.\nFotografije snimljene sigurnosnom kamerom pokazale su na koji se način zgrada srušila na prometnu cestu.\nU Turskoj je nakon potresa ostalo gotovo dva milijuna beskućnika. Mnogi su smješteni u šatorske kampove, no dio ljudi se počeo vraćati u svoje oštećene domove."
27938,"Ovo je pravo čudo: čak 21 dan nakon razornog potresa u Turskoj, iz ruševina je spašen - živ i zdrav konj","Od razornog potresa koji je pogodio Tursku i Siriju prošao je već 21 dan, no vijesti o čudesnim spašavanjima još pristižu.\nToliko vrijeme pod ruševinama ne bi mogao preživjeti niti jedan čovjek, no preživio je jedan - konj.\nTursko ministarstvo poljoprivrede i šumarstva objavilo je video iz grada Adıyaman gdje je konj izvučen iz ruševina.\nUz malu pomoć, konj se sam uspeo na čvrto tlo i odmah je odveden veterinaru koji je ustanovio da nema teže ozljede.\nSretni konj je nakon toga vraćen vlasniku."
20267,Europska komisija: Dajemo milijardu eura Turskoj za pomoć nakon potresa,"OBJAVILA VON DER LEYEN\nEuropska komisija: Dajemo milijardu eura Turskoj za pomoć nakon potresa\nEuropska komisija dat će milijardu eura pomoći potresom pogođenoj Turskoj, rekla je u ponedjeljak predsjednica tog tijela Ursula von der Leyen na početku međunarodne donors

## BM25

In [43]:
!pip install rank-bm25

In [44]:
nlp = spacy.load("hr_core_news_lg")
corpus = []
for i, data in tqdm(df.iterrows()):
    corpus.append([i.lemma_ for i in nlp(data["body"]) if (i not in nlp.Defaults.stop_words) and (not i.is_punct)])
corpus = pd.DataFrame.from_dict({"lemmas": corpus})

29625it [22:25, 22.02it/s]


In [45]:
corpus["lemmas"] = corpus["lemmas"].apply(
    lambda x: [i for i in x if i!="\n"]
)
print(corpus.shape)
corpus.head()

(29625, 1)


,lemmas
0,"[Električan, problem, zaustaviti, biti, u, utorak, nešto, prije, 10, sat, tramvajski, promet, između, Višnjevac, i, zeleni, polje, kako, biti, biti, struja, koji, biti, pokrenuti, tramvaj, iz, gradski, prijevoz, putnik, u, promet, biti, odmah, nakon, saznanje, za, kvar, u, promet, pustiti, četiri, autobus, Putnik, koji, biti, sebe, zateći, u, tramvaj, do, svoj, odredište, potom, biti, doći, autobus, iz, GPP, a, poručivati, kako, biti, u, svaki, trenutak, prilikom, ovakav, situacija, spreman, organizirati, alternativan, prijevoz, a, sve, kako, biti, građanin, na, vrijeme, stići, do, željen, destinacija, M.Bu]"
1,"[Most, Bir, Hakeim, koji, biti, poznat, po, željezan, stupov, i, pogled, na, eiffelov, toranj, preimenovati, htjeti, sebe, u, Šetalište, Jean, paula, Belmond, u, znak, sjećanje, na, francuski, glumac, koji, biti, umrijeti, 2021, objaviti, biti, vijeće, grad, Pariz, on, biti, ondje, snimiti, kultan, scena, iz, krimić, Henrij, Verneuila, peur, sur, la, Ville, iz, 1975, u, koji, glumiti, policajac, Jean, Letellier, Tragajući, za, serijski, ubojic, koji, progoniti, mlad, žena, u, jedan, scena, završiti, i, na, krov, vagon, metro, linija, broj, 6, koji, u, jedan, dio, put, prelaziti, preko, Seina, ispod, most, Bir, Hakeim, Političarka, i, vijećnica, Maud, Gatel, nekoliko, biti, tjedan, nakon, ...]"
2,"[Njemačka, biti, u, 2022, povećati, broj, javan, punionica, za, električan, vozilo, za, 35, posto, na, 80.500, izvijestiti, biti, Njemačko, udruženje, industrija, energija, i, voda, BDEW, nov, punionica, povećati, biti, kapacitet, za, punjenje, za, mnogo, od, 40, posto, na, 2,47, gigavat, navoditi, BDEW, snažno, povećanje, kapacitet, udruga, objašnjavati, u, prvi, red, znatno, velik, broj, punkt, za, ultrabrz, punjenje, kapacitet, od, malo, 150, kilovat, broj, punionica, često, biti, meta, kritika, argumentiraan, vladin, plan, o, milijun, punionica, do, 2030, navoditi, Andreae, tvrditi, da, biti, taj, cilj, zapravo, tehnički, zastariti, budući, da, ne, uzimati, u, obzir, kapacitet, za, punjenje, Razgranata, mreža, punionica, bez, pogovor, ...]"
3,"[Aramco, globalan, energetsko, kemijska, tvrtka, potpisati, biti, pismo, namjera, sa, proizvođač, automobil, Geely, i, Renault, Grupa, radi, stvaranje, nov, tvrtka, za, razvoj, pogonski, tehnologija, sa, nisk, razina, emisija, riječ, biti, o, razvoj, rješenje, u, područje, sintetički, gorivo, te, razvoj, nov, generacija, tehnologija, na, temelj, vodik, sa, globalan, mreža, od, 17, tvornica, za, proizvodnja, motor, i, pet, znanstveno, istraživačkih, centar, na, tri, kontinent, nov, biti, tvrtka, trebati, postati, samostalan, globalan, dobavljač, osim, taj, očekivati, sebe, da, htjeti, godišnje, proizvoditi, mnogo, od, pet, milijun, termički, hibridni, i, plug, in, hibridni, motor, i, prijenosan, sustav, te, opskrbljivati, mnogo, od, 130, zemlja, i, regija, ...]"
4,"[istraživanje, o, korištenje, sigurnosni, pojas, koji, biti, predstaviti, Roadpol, pokazati, biti, da, vozač, kombije, i, minibuseva, daleko, zaostati, u, pridržavanje, pravilo, u, odnos, na, vozač, osoban, automobil, iako, ni, kod, potonji, nemati, poboljšanje, K, taj, čak, 40, dijete, putovati, u, automobilo, na, nesiguran, način, pokazati, biti, nedavan, istraživanje, Roadpola, europski, mreža, prometan, policija, također, biti, zaključiti, da, biti, vikend, vozači, oprezan, od, svakodnevan, vozač, dok, biti, u, poglavlje, o, ne)vezivanje, sigurnosni, pojas, jedan, kategorija, posebno, problematičan, istraživanje, biti, tako, pokazati, da, vozač, kombije, i, minibuseva, daleko, zaostati, u, pridržavanje, slovo, zakon, u, usporedba, sa, vozač, osoban, automobil, u, istraživanje, provedenom, ...]"


In [46]:
corpus.to_csv("../data/textual_similarity_lemmas.csv", index=False)

In [47]:
tokenized_corpus = corpus["lemmas"].values.tolist()
print(tokenized_corpus[15])

['Tim', 'zadužiti', 'za', 'razvoj', 'i', 'testiranje', 'Mercedes', 'Benz', 'Trucks', 'testirati', 'biti', 'operativan', 'spremnost', 'eActros', 'LongHaul', 'i', 'eActros', '300', 'u', 'ekstreman', 'uvjet', 'kako', 'biti', 'provjeriti', 'njihov', 'ponašanje', 'i', 'mogućnost', 'te', 'izvršiti', 'eventualan', 'poboljšanje', 'naime', 'pred', 'električan', 'kamion', 'postavljati', 'sebe', 'jednak', 'zahtjev', 'kao', 'pred', 'kamion', 'sa', 'klasičan', 'pogonski', 'sustav', 'pogotovo', 'kad', 'biti', 'riječ', 'o', 'težak', 'vremenski', 'uvjet', 'kao', 'što', 'biti', 'hladnoća', 'snijeg', 'i', 'led', 'ovogodišnji', 'zimski', 'test', 'obavljeni', 'biti', 'u', 'Rovaniemija', 'u', 'Finska', 'u', 'trajanje', 'od', 'šest', 'tjedan', 'i', 'pokazati', 'biti', 'sebe', 'kao', 'vrlo', 'važan', 'test', 'izdržljivost', 'Testnim', 'ciklus', 'obuhvaćeni', 'biti', 'različit', 'kamion', 'tegljač', 'različit', 'modelski', 'serija', 'uključivati', 'prototip', 'električan', 'tegljač', 'eActros', 'LongHaul', 'M

In [48]:
from rank_bm25 import BM25Okapi

bm25 = BM25Okapi(tokenized_corpus)
bm25

In [49]:
query = " ".join(tokenized_corpus[20])
print(query)
print(" -------------------------------------------------------------------------------------------------------------------- ")

res = bm25.get_top_n(query.split(" "), tokenized_corpus, n=10)[1:]
for i, r in enumerate(res):
    print(f"{i}: {' '.join(r)}")
    print()

europski komisija dati htjeti milijarda euro pomoći potres pogođeni Turska reći biti u ponedjeljak predsjednica taj tijelo Ursula von der Leyen na početak međunarodni donorski konferencija Von der Leyen biti o potres koji biti odniti gotovo 50 tisuća život u Turska i Sirija govoriti kao o najveći prirodan katastrofa koji biti pogoditi regija u posljednji nekoliko godina Kuće škola i bolnica morati sebe ponovno izgraditi uz mnogo standard sigurnost od potres Voda kanalizacija i drugi ključan infrastruktura morati sebe popraviti poručiti biti europski komisija dati htjeti dodatan 108 milijun euro za humanitaran pomoć i oporavak Sirija H
 -------------------------------------------------------------------------------------------------------------------- 
0: europski komisija dati htjeti milijarda euro pomoći potres pogođeni Turska reći biti u ponedjeljak predsjednica taj tijelo Ursula von der Leyen na početak međunarodni donorski konferencija 20.03.2023 15:49 Autor M.M./Hina HRT europski 

In [50]:
df.iloc[[24997, 20267]][["body"]]

,body
25337,"Hrvatski tenisač Matej Dodig u nedjelju će se na ITF turniru u Rovinju (15.000 USD) boriti za svoj prvi naslov u seniorskoj konkurenciji, a mjesto u finalu izborio je subotnjom pobjedom 6-2, 6-3 protiv sedmog nositelja, 20-godišnjeg Talijana Fausta Tabacca (ATP - 502.).\nDodigov suparnik u finalu bit će još jedan talijanski predstavnik, 22-godišnji šesti nositelj Federico Arnaboldi (ATP - 479.) koji je svladao Rumunja Cezara Cretua sa 6-2, 6-4.\nDodig je za pobjedu protiv Tabacca trebao 83 minute, a u drugom setu je nadoknadio rani ""break"" zaostatka, da bi od rezultata 3-3 nanizao tri gema za pobjedu i plasman u finale.\nZa 17-godišnjeg Osječanina ovo je tek drugi ITF turnir u seniorskoj konkurenciji, što njegov rezultat čini još značajnijim. Dodig je prošli tjedan u Poreču osvojio svoj prvi ATP bod, a ulaskom u finale osigurao ih je još osam. Osvajanje naslova pobjedniku donosi 15 bodova i novčanu nagradu od 1944 eura.\nArnaboldi iza sebe ima već tri osvojena trofeja na ITF Touru, a sva tri je osvojio na zemljanoj podlozi, na kakvoj se igra i u rovinjskom TC ""Eden"", domaćinu drugog od tri turnira iz serije Istarska rivijera.\nSljedeći tjedan je na rasporedu posljednji turnir 51. izdanja Istarske rivijere, a igrat će se na terenima TK ""Opatija"".\nVijesti HRT-a pratite na svojim pametnim telefonima i tabletima putem aplikacija za iOS i Android. Pratite nas i na društvenim mrežama Facebook, Twitter, Instagram i YouTube!"
20471,"POŽALILA SE NA TWITTERU\nAnđa Marić pobjesnila zbog birokracije: Je*** vam mater. Morala sam dokazivati da imam multiplu, ukinuli su mi doplatak\nGlazbenica i spisateljica Anđa Marić, koja se bori s multiplom sklerozom, na društvenim je mrežama pisala o birokratskoj muci s kojom se suočava.\n""Iako sam imala invaliditet 80 posto 2018. godine, ove godine su mi ukinuli doplatak od 600 kuna mjesečno. Odbili su mi zahtjev za osobnu invalidninu koja je 225 eura jer im izgledam dobro pa mi valjda ne treba. Samo me treninzi i taksi koštaju mjesečno oko 3000 kuna"", napisala je na Twitteru.\niako sam imala invaliditet 80% 2018. godine ove godine su mi ukinuli doplatak od 600 kn mjesečno. odbili su mi zahtjev za osobnu invalidninu koja je 225 € jer im izgledam dobro pa mi valjda ne treba. Samo me treninzi i taxi koštaju mjesečno oko 3000 kn— Anđa Marić (@AndjaMarics) March 16, 2023\n""Danas me zovu da mi je ipak odobren doplatak koji mi je ukinut prošle godine u 12. mjesecu jer sam morala doći dokazati da još uvijek imam multiplu sklerozu iako me nisu zvali na vještačenje i nisu dizali telefon i odgovarali na mail od 12. mjeseca"", nastavlja iznositi detalje na Twitteru.\n""I pravnik socijalne Trešnjevka mi je rekao da, sudeći po aplikaciji koju on ima, ne vidi da mi je ikada ukinut doplatak i da jer nažalost to naša država... Je*em vam mater, vi ste država, ja sam država i svi smo mi država. Tko mi kriv što sam pozitivna i ne zaslužujem pomoć i potporu"", nastavlja.\n""OK, ispuhala sam se, sada idemo dalje bez cendranja. Nekad i meni prekipi iako sam okorjela pozitivka. To stanje biram namjerno svaki dan iako i ja doživljavam sve što i vi, ali da mi je pažnja na onom što mi se ne sviđa – već bih davno poludjela"", napisala je Marić te se ispričala pratiteljima na trenutku očaja i brutalne iskrenosti.\n""‘Ispričavam se što sam svoj Twitter danas pretvorila u septičku jamu. Ljudi, dolazi nam proljeće, još samo tri menstruacije do ljeta"", zaključila je."


In [51]:
df.to_csv("../data/textual_similarity_recent_no_dups.csv", index=False)